In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from torch.utils.data import DataLoader, TensorDataset

# 读取训练集和测试集数据
data_path = '/bohr/Superconductor-data-bn10/v1/'
train_data = pd.read_csv(f'{data_path}superconductor-train.csv')
test_data = pd.read_csv(f'{data_path}superconductor-test-no-label.csv')

# 特征和目标值
X = train_data.drop('y', axis=1)
y = train_data['y']

# 将训练数据划分为训练集和验证集
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 对数据进行标准化处理
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
test_data = scaler.transform(test_data)

##########################################
## 完善代码1：以上是使用StandardScaler()库函数实例化的scaler；试着不使用库函数，自行写代码对数据进行标准化
###########################################

##########################################
## 特征工程介绍
## 1.特征降维：例如使用PCA（主成分分析）等方法将高维的数据映射到低维度
## 2.特征变换：不改变特征的维度，在原特征上施加线性（例如标准化和归一化）或者非线性变换（例如对数变换等）
## 3.特征筛选：保留一些重要的特征，删除对结果影响较小或没有影响（思考：什么样会没有影响）的特征列
## 4.特征设计：手动或自动设计、增加特征
## 5.其他方法，大胆尝试（尤其是选择此课题为扩展部分展示的同学）
###########################################

##########################################
## 完善代码2：除以上的标准化外，自己任意选择一种特征工程方法实现，并对比前后分数结果（不一定要有提高）
###########################################

# 将数据转换为PyTorch张量
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).view(-1, 1)
test_tensor = torch.tensor(test_data, dtype=torch.float32)

# 创建数据加载器
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

class SuperconductorModel(nn.Module):
    ##########################################
    ## 完善代码3：修改神经网络架构、参数，例如加深层数等
    ###########################################
    def __init__(self):
        super(SuperconductorModel, self).__init__()
        self.layer1 = nn.Linear(86, 128)
        self.layer2 = nn.Linear(128, 64)
        self.output = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        return self.output(x)

model = SuperconductorModel()

# 使用均方误差作为损失函数
criterion = nn.MSELoss()

# 使用Adam优化器
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练模型
num_epochs = 1000
best_val_loss = 1000
for epoch in range(num_epochs):
    model.train()  # 设置模型为训练模式
    for batch_X, batch_y in train_loader:
        # 前向传播
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 每个epoch结束后评估模型在验证集上的表现
    model.eval()  # 设置模型为评估模式
    with torch.no_grad():
        output = model(X_val_tensor)
        val_loss = criterion(output, y_val_tensor)
    
    avg_val_loss = val_loss.item()
    if avg_val_loss < best_val_loss:
        ##########################################
        ## 完善代码4：更新best_val_loss并且记录最佳模型参数为best_model
        ###########################################
    
    print(f'Epoch {epoch+1}/{num_epochs}, Validation Loss: {avg_val_loss:.4f}, Best Loss: {best_val_loss:.4f}')

model.load_state_dict(best_model)

model.eval()
with torch.no_grad():
    y_val_pred = model(X_val_tensor)

# 计算误差
mae = mean_absolute_error(y_val, y_val_pred)
print(f"Mean Absolute Error: {mae}")

# 预测测试集的值
model.eval()
with torch.no_grad():
    test_predictions = model(test_tensor)

# 将预测结果保存为csv文件
pd.DataFrame(test_predictions.numpy(), columns=['test_pre_y']).to_csv('submission.csv', index=False)